# 🏆 Ranking e Certificação de Endereços por CPF - Ouro

Este notebook:
1. Carrega endereços normalizados da camada Prata
2. Agrupa endereços por CPF
3. Calcula score de confiabilidade para cada endereço
4. Cria ranking de endereços por CPF
5. Calcula percentual de probabilidade de ser o endereço atual
6. Marca o endereço com maior score como **certificado**
7. Salva resultados na camada Ouro

## 📊 Critérios de Score:
- **Completude** (peso 40%): Endereço completo tem maior probabilidade
- **Frequência** (peso 30%): Endereços que aparecem mais vezes
- **Qualidade** (peso 20%): CEP válido, UF válida, dados consistentes
- **Ordem** (peso 10%): Primeiro endereço pode ser mais importante

## 1. Configuração e Importação

In [ ]:
import pandas as pd
import numpy as np
from minio import Minio
import io
from datetime import datetime
import sys
import os
import warnings
warnings.filterwarnings('ignore')

# Configurações MinIO
MINIO_SERVER_URL = "ch8ai-minio.l6zv5a.easypanel.host"
MINIO_ROOT_USER = "admin"
MINIO_ROOT_PASSWORD = "1q2w3e4r"
BUCKET_NAME = "govbr"

print("✅ Bibliotecas importadas com sucesso!")

## 2. Conexão com MinIO

In [ ]:
# Conectar ao MinIO
minio_client = Minio(
    MINIO_SERVER_URL,
    access_key=MINIO_ROOT_USER,
    secret_key=MINIO_ROOT_PASSWORD,
    secure=True
)

print("✅ Conectado ao MinIO")

## 3. Funções Auxiliares

In [ ]:
def read_from_prata(dataset_name, partition_date=None):
    """Lê DataFrame da camada Prata"""
    if partition_date is None:
        # Buscar a partição mais recente
        prefix = f"prata/{dataset_name}/"
        objects = list(minio_client.list_objects(BUCKET_NAME, prefix=prefix, recursive=True))
        if not objects:
            return None
        
        # Pegar o mais recente
        latest = max(objects, key=lambda x: x.last_modified)
        object_name = latest.object_name
    else:
        object_name = f"prata/{dataset_name}/dt={partition_date}/data.parquet"
    
    try:
        response = minio_client.get_object(BUCKET_NAME, object_name)
        df = pd.read_parquet(io.BytesIO(response.read()))
        response.close()
        response.release_conn()
        print(f"✅ Lido Prata: {object_name} ({len(df)} registros)")
        return df
    except Exception as e:
        print(f"❌ Erro ao ler {object_name}: {e}")
        return None

def save_to_ouro(df, dataset_name, partition_date=None):
    """Salva DataFrame na camada Ouro em formato Parquet"""
    if partition_date is None:
        partition_date = datetime.now().strftime('%Y%m%d')
    
    object_name = f"ouro/{dataset_name}/dt={partition_date}/data.parquet"
    
    try:
        buffer = io.BytesIO()
        df.to_parquet(buffer, index=False, engine='pyarrow', compression='snappy')
        buffer.seek(0)
        
        minio_client.put_object(
            BUCKET_NAME,
            object_name,
            buffer,
            length=buffer.getbuffer().nbytes,
            content_type='application/octet-stream'
        )
        
        tamanho_mb = buffer.getbuffer().nbytes / 1024 / 1024
        print(f"✅ Ouro: {object_name}")
        print(f"   • {len(df):,} registros")
        print(f"   • {tamanho_mb:.2f} MB")
        return True
    except Exception as e:
        print(f"❌ Erro ao salvar {object_name}: {e}")
        return False

print("✅ Funções auxiliares definidas!")

## 4. Carregar Dados da Camada Prata

In [ ]:
# Carregar endereços normalizados da Prata
print("📂 Carregando dados da camada Prata...")
print("=" * 80)

df_prata = read_from_prata('cidadaos_enderecos_normalizados')

if df_prata is not None:
    print(f"\n✅ Dados carregados: {len(df_prata):,} registros")
    print(f"   • CPFs únicos: {df_prata['cpf'].nunique():,}")
    print(f"   • Endereços únicos: {df_prata['endereco_normalizado'].nunique():,}")
    
    print(f"\n📊 Estrutura dos dados:")
    print(f"   • Colunas: {list(df_prata.columns)}")
    
    # Mostrar amostra
    print(f"\n📝 Amostra dos dados (primeiras 3 linhas):")
    display(df_prata[['cpf', 'nome', 'endereco_normalizado', 'completo', 'numero_endereco']].head(3))
else:
    print("⚠️  Nenhum dado encontrado na camada Prata.")
    print("   Execute primeiro o notebook de normalização.")

## 5. Calcular Score de Confiabilidade

In [ ]:
def calcular_score_endereco(row):
    """Calcula score de confiabilidade de um endereço"""
    score = 0.0
    
    # 1. Completude (peso 40%)
    if row['completo']:
        score += 40.0
    else:
        # Pontuação parcial baseada em componentes presentes
        componentes_presentes = sum([
            row['tem_cep'],
            row['tem_uf'],
            row['tem_municipio'],
            row['tem_bairro'],
            row['tem_complemento']
        ])
        score += (componentes_presentes / 5) * 40.0
    
    # 2. Qualidade dos dados (peso 20%)
    qualidade_score = 0.0
    if pd.notna(row['cep']) and len(str(row['cep']).replace('-', '')) == 8:
        qualidade_score += 5.0  # CEP válido
    if pd.notna(row['uf']) and len(str(row['uf'])) == 2:
        qualidade_score += 5.0  # UF válida
    if pd.notna(row['municipio']) and len(str(row['municipio'])) > 2:
        qualidade_score += 5.0  # Município válido
    if pd.notna(row['numero_imovel']):
        qualidade_score += 5.0  # Número presente
    score += qualidade_score
    
    # 3. Ordem do endereço (peso 10%)
    # Primeiro endereço (numero_endereco = 1) recebe pontuação máxima
    if pd.notna(row['numero_endereco']):
        if row['numero_endereco'] == 1:
            score += 10.0
        elif row['numero_endereco'] <= 3:
            score += 7.0
        elif row['numero_endereco'] <= 7:
            score += 4.0
        else:
            score += 1.0
    
    # 4. Frequência será calculada depois (peso 30%)
    # Será adicionado no agrupamento por CPF
    
    return score

print("✅ Função de cálculo de score definida!")

In [ ]:
if df_prata is not None:
    print("🔄 Calculando scores iniciais...")
    print("=" * 80)
    
    # Calcular score base (sem frequência)
    df_prata['score_base'] = df_prata.apply(calcular_score_endereco, axis=1)
    
    print(f"✅ Scores base calculados!")
    print(f"   • Score médio: {df_prata['score_base'].mean():.2f}")
    print(f"   • Score mínimo: {df_prata['score_base'].min():.2f}")
    print(f"   • Score máximo: {df_prata['score_base'].max():.2f}")
    
    # Mostrar distribuição
    print(f"\n📊 Distribuição de scores:")
    print(df_prata['score_base'].describe())

## 6. Agrupar por CPF e Calcular Frequência

In [ ]:
if df_prata is not None:
    print("🔄 Agrupando endereços por CPF e calculando frequência...")
    print("=" * 80)
    
    # Criar chave única para endereço normalizado
    df_prata['chave_endereco'] = df_prata['endereco_normalizado'].fillna('')
    
    # Contar frequência de cada endereço por CPF
    freq_enderecos = df_prata.groupby(['cpf', 'chave_endereco']).size().reset_index(name='frequencia')
    
    # Calcular frequência relativa (percentual dentro do CPF)
    freq_relativa = df_prata.groupby('cpf').size().reset_index(name='total_enderecos_cpf')
    freq_enderecos = freq_enderecos.merge(freq_relativa, on='cpf')
    freq_enderecos['frequencia_relativa'] = (freq_enderecos['frequencia'] / freq_enderecos['total_enderecos_cpf']) * 100
    
    # Merge de volta
    df_prata = df_prata.merge(
        freq_enderecos[['cpf', 'chave_endereco', 'frequencia', 'frequencia_relativa']],
        on=['cpf', 'chave_endereco'],
        how='left'
    )
    
    print(f"✅ Frequência calculada!")
    print(f"   • Média de frequência: {df_prata['frequencia'].mean():.2f}")
    print(f"   • Média de frequência relativa: {df_prata['frequencia_relativa'].mean():.2f}%")
    
    # Mostrar exemplo
    print(f"\n📝 Exemplo de frequência:")
    exemplo_cpf = df_prata['cpf'].iloc[0]
    exemplo_df = df_prata[df_prata['cpf'] == exemplo_cpf][
        ['cpf', 'endereco_normalizado', 'frequencia', 'frequencia_relativa']
    ].drop_duplicates()
    display(exemplo_df.head())

## 7. Calcular Score Final e Ranking

In [ ]:
if df_prata is not None:
    print("🔄 Calculando score final e criando ranking...")
    print("=" * 80)
    
    # Adicionar pontuação de frequência (peso 30%)
    # Normalizar frequência relativa para 0-30 pontos
    df_prata['score_frequencia'] = (df_prata['frequencia_relativa'] / 100) * 30.0
    
    # Score final = score_base + score_frequencia
    df_prata['score_final'] = df_prata['score_base'] + df_prata['score_frequencia']
    
    # Criar ranking por CPF
    df_prata['ranking_cpf'] = df_prata.groupby('cpf')['score_final'].rank(
        method='dense',
        ascending=False
    ).astype(int)
    
    # Calcular percentual de probabilidade
    # Soma dos scores por CPF
    soma_scores_cpf = df_prata.groupby('cpf')['score_final'].transform('sum')
    df_prata['percentual_probabilidade'] = (df_prata['score_final'] / soma_scores_cpf) * 100
    
    # Marcar endereço certificado (ranking = 1)
    df_prata['endereco_certificado'] = df_prata['ranking_cpf'] == 1
    
    print(f"✅ Ranking criado!")
    print(f"   • Score final médio: {df_prata['score_final'].mean():.2f}")
    print(f"   • Score final máximo: {df_prata['score_final'].max():.2f}")
    print(f"   • Endereços certificados: {df_prata['endereco_certificado'].sum():,}")
    print(f"   • Percentual médio de probabilidade: {df_prata['percentual_probabilidade'].mean():.2f}%")
    
    # Estatísticas do ranking
    print(f"\n📊 Estatísticas do ranking:")
    ranking_stats = df_prata.groupby('ranking_cpf').agg({
        'score_final': 'mean',
        'percentual_probabilidade': 'mean',
        'cpf': 'count'
    }).rename(columns={'cpf': 'quantidade'})
    print(ranking_stats.head(10))

## 8. Preparar Dados para Camada Ouro

In [ ]:
if df_prata is not None:
    print("🔄 Preparando dados para camada Ouro...")
    print("=" * 80)
    
    # Selecionar colunas relevantes para Ouro
    colunas_ouro = [
        # Dados do cidadão
        'cpf',
        'nome',
        'telefone',
        'tipo_telefone',
        'email',
        'total_enderecos',
        
        # Endereço normalizado
        'endereco_normalizado',
        'endereco_original',
        
        # Componentes estruturados
        'tipo_logradouro',
        'nome_logradouro',
        'numero_imovel',
        'complemento',
        'bairro',
        'municipio',
        'uf',
        'cep',
        
        # Qualidade
        'completo',
        'tem_complemento',
        'tem_bairro',
        'tem_municipio',
        'tem_uf',
        'tem_cep',
        
        # Ranking e certificação
        'numero_endereco',
        'ranking_cpf',
        'score_base',
        'score_frequencia',
        'score_final',
        'percentual_probabilidade',
        'endereco_certificado',
        'frequencia',
        'frequencia_relativa'
    ]
    
    df_ouro = df_prata[colunas_ouro].copy()
    
    # Ordenar por CPF e ranking
    df_ouro = df_ouro.sort_values(['cpf', 'ranking_cpf'])
    
    print(f"✅ Dados preparados!")
    print(f"   • Total de registros: {len(df_ouro):,}")
    print(f"   • CPFs únicos: {df_ouro['cpf'].nunique():,}")
    print(f"   • Endereços certificados: {df_ouro['endereco_certificado'].sum():,}")
    
    # Mostrar amostra
    print(f"\n📝 Amostra dos dados (primeiros 5 registros):")
    display(df_ouro[
        ['cpf', 'nome', 'endereco_normalizado', 'ranking_cpf', 
         'score_final', 'percentual_probabilidade', 'endereco_certificado']
    ].head(5))

## 9. Análise de Endereços Certificados

In [ ]:
if df_ouro is not None:
    print("📊 Análise de Endereços Certificados")
    print("=" * 80)
    
    # Filtrar apenas endereços certificados
    df_certificados = df_ouro[df_ouro['endereco_certificado']].copy()
    
    print(f"\n✅ Estatísticas dos Endereços Certificados:")
    print(f"   • Total de endereços certificados: {len(df_certificados):,}")
    print(f"   • Score médio: {df_certificados['score_final'].mean():.2f}")
    print(f"   • Percentual médio: {df_certificados['percentual_probabilidade'].mean():.2f}%")
    print(f"   • Endereços completos: {df_certificados['completo'].sum():,} ({df_certificados['completo'].sum()/len(df_certificados)*100:.1f}%)")
    
    # Distribuição por UF
    print(f"\n🗺️ Distribuição por Estado (Top 10):")
    uf_dist = df_certificados['uf'].value_counts().head(10)
    for uf, qtd in uf_dist.items():
        if pd.notna(uf):
            percentual = (qtd / len(df_certificados)) * 100
            print(f"   • {uf}: {qtd:,} ({percentual:.2f}%)")
    
    # Distribuição de percentuais
    print(f"\n📊 Distribuição de Percentuais de Probabilidade:")
    print(df_certificados['percentual_probabilidade'].describe())
    
    # Exemplos de endereços certificados
    print(f"\n📝 Exemplos de Endereços Certificados:")
    exemplos = df_certificados.head(5)
    for idx, row in exemplos.iterrows():
        print(f"\n   CPF: {row['cpf']}")
        print(f"   Nome: {row['nome']}")
        print(f"   Endereço: {row['endereco_normalizado']}")
        print(f"   Score: {row['score_final']:.2f}")
        print(f"   Probabilidade: {row['percentual_probabilidade']:.2f}%")
        print(f"   Ranking: {row['ranking_cpf']}")
        print("-" * 80)

## 10. Visualizar Ranking por CPF

In [ ]:
if df_ouro is not None:
    print("🔍 Visualizando Ranking por CPF")
    print("=" * 80)
    
    # Selecionar alguns CPFs para exemplo
    cpfs_exemplo = df_ouro['cpf'].unique()[:5]
    
    for cpf in cpfs_exemplo:
        df_cpf = df_ouro[df_ouro['cpf'] == cpf].copy()
        df_cpf = df_cpf.sort_values('ranking_cpf')
        
        print(f"\n📋 CPF: {cpf}")
        print(f"   Nome: {df_cpf['nome'].iloc[0]}")
        print(f"   Total de endereços: {len(df_cpf)}")
        print(f"\n   Ranking de Endereços:")
        
        for idx, row in df_cpf.iterrows():
            certificado = "✅ CERTIFICADO" if row['endereco_certificado'] else ""
            print(f"\n   [{row['ranking_cpf']}] {certificado}")
            print(f"      Endereço: {row['endereco_normalizado']}")
            print(f"      Score: {row['score_final']:.2f}")
            print(f"      Probabilidade: {row['percentual_probabilidade']:.2f}%")
            print(f"      Frequência: {row['frequencia']} ({row['frequencia_relativa']:.1f}%)")
            print(f"      Completo: {'✅' if row['completo'] else '❌'}")
        
        print("\n" + "=" * 80)

## 11. Salvar na Camada Ouro

In [ ]:
if df_ouro is not None:
    print("💾 Salvando dados na camada Ouro...")
    print("=" * 80)
    
    # Salvar na camada Ouro
    sucesso = save_to_ouro(df_ouro, 'cidadaos_enderecos_rankings')
    
    if sucesso:
        print(f"\n✅ Dados salvos com sucesso na camada Ouro!")
        print(f"\n📁 Localização: ouro/cidadaos_enderecos_rankings/dt=YYYYMMDD/data.parquet")
        
        # Verificar arquivo salvo
        print(f"\n🔍 Verificando arquivo salvo...")
        partition_date = datetime.now().strftime('%Y%m%d')
        object_name = f"ouro/cidadaos_enderecos_rankings/dt={partition_date}/data.parquet"
        
        try:
            response = minio_client.stat_object(BUCKET_NAME, object_name)
            tamanho_mb = response.size / 1024 / 1024
            print(f"   ✅ Arquivo confirmado: {tamanho_mb:.2f} MB")
        except Exception as e:
            print(f"   ⚠️  Erro ao verificar arquivo: {e}")
    else:
        print(f"\n❌ Erro ao salvar dados.")
else:
    print("⚠️  Nenhum dado para salvar.")

## 12. Resumo Final

In [ ]:
if df_ouro is not None:
    print("=" * 80)
    print("📋 RESUMO DO RANKING E CERTIFICAÇÃO")
    print("=" * 80)
    
    print(f"\n📊 Estatísticas Gerais:")
    print(f"   • Total de registros processados: {len(df_ouro):,}")
    print(f"   • CPFs únicos: {df_ouro['cpf'].nunique():,}")
    print(f"   • Endereços únicos: {df_ouro['endereco_normalizado'].nunique():,}")
    
    print(f"\n🏆 Ranking:")
    print(f"   • Endereços certificados: {df_ouro['endereco_certificado'].sum():,}")
    print(f"   • Score médio (certificados): {df_ouro[df_ouro['endereco_certificado']]['score_final'].mean():.2f}")
    print(f"   • Percentual médio (certificados): {df_ouro[df_ouro['endereco_certificado']]['percentual_probabilidade'].mean():.2f}%")
    
    print(f"\n✅ Qualidade dos Endereços Certificados:")
    certificados = df_ouro[df_ouro['endereco_certificado']]
    print(f"   • Completos: {certificados['completo'].sum():,} ({certificados['completo'].sum()/len(certificados)*100:.1f}%)")
    print(f"   • Com CEP: {certificados['tem_cep'].sum():,} ({certificados['tem_cep'].sum()/len(certificados)*100:.1f}%)")
    print(f"   • Com UF: {certificados['tem_uf'].sum():,} ({certificados['tem_uf'].sum()/len(certificados)*100:.1f}%)")
    print(f"   • Com município: {certificados['tem_municipio'].sum():,} ({certificados['tem_municipio'].sum()/len(certificados)*100:.1f}%)")
    
    print(f"\n📁 Dados Salvos:")
    partition_date = datetime.now().strftime('%Y%m%d')
    print(f"   • Camada: Ouro")
    print(f"   • Dataset: cidadaos_enderecos_rankings")
    print(f"   • Partição: dt={partition_date}")
    
    print(f"\n✅ Processo concluído com sucesso!")